## 外れ値が多かったり, Loyalityに特徴があるCity, State, Subsectorを特徴にする

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

feat_no = '115_out_'

### Data Load

In [2]:
hist = utils.read_df_pkl('../input/historical_transactions0*')
new = utils.read_df_pkl('../input/new*0*')
df = pd.concat([hist, new], axis=0)

100%|██████████| 3/3 [00:02<00:00,  1.40it/s]


### City_id

In [39]:
#========================================================================
# City_id
feat_col = 'city_id'
f1 = pd.read_csv('../input/city_not33_mean_cnt_id.csv')
f2 = pd.read_csv('../input/city_not33_mean_purchase_amount_mean.csv')
f3 = pd.read_csv('../input/city_ratio_out_cnt_id.csv')
# 1
# l1 = f1[f1.target_mean>-0.07][feat_col].values.tolist()
# 2
# l2 = f1[f1.target_mean<-0.17][feat_col].values.tolist()
# 3
# l3 = f2[f2.target_mean>-0.07][feat_col].values.tolist()
# 4
# l4 = f2[f2.target_mean<-0.17][feat_col].values.tolist()
# 5
# l5 = f3[f3.ratio_out>0.0075][feat_col].values.tolist()
# 6
# l6 = f3[f3.ratio_out<0.005][feat_col].values.tolist()

# feat_list = list(set(l1 + l2 + l3 + l4 + l5 + l6))
feat_list = list(set(l5 + l6))
#========================================================================

# Squeeze City
df_m = df.loc[df[feat_col].isin(feat_list), [feat_col, 'card_id']]
print(df_m.shape)
df_m.head()

# One Hot
tmp = df_m.drop_duplicates().set_index(key)
tmp['value'] = 1
ohe = tmp.pivot_table(index=key, columns=feat_col, values='value', aggfunc='max').fillna(0).astype('int')

for col in ohe.columns:
    ohe.rename(columns={col:f"{feat_col}_{col}"}, inplace=True)

base = utils.read_df_pkl('../input/base*')
base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)

df_train = base_train.merge(ohe, how='left', on=key)
df_test = base_test.merge(ohe, how='left', on=key)

for col in df_train.columns:
    if col in ignore_list:continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/115_out_train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/115_out_test_{col}@', obj=df_test[col].values)

(6396741, 2)


100%|██████████| 3/3 [00:00<00:00, 84.34it/s]


### State_id

In [40]:
#========================================================================
# State_id
feat_col = 'state_id'
f4 = pd.read_csv('../input/state_ratio_out_cnt_id.csv')

# 1
l1 = f4[f4.ratio_out<0.005][feat_col].values.tolist()

# feat_list = list(set(l1 + l2 + l3 + l4 + l5 + l6))
feat_list = list(set(l5 + l6))
#========================================================================

# Squeeze City
df_m = df[[feat_col, 'card_id']].copy()

# One Hot
df_m['value'] = 1
ohe = df_m.pivot_table(index=key, columns=feat_col, values='value', aggfunc='sum').fillna(0).astype('int')

ohe.head()

   state_id  cnt_id  ratio_out  state_cnt_id
0        -1  103783   0.006190        167432
1         1    5911   0.006480          9509
2         2    7941   0.006948         12864
3         3   10209   0.005948         16572
4         4   15750   0.006825         25424


SystemExit: 

In [37]:
for col in ohe.columns:
    ohe.rename(columns={col:f"{feat_col}_{col}"}, inplace=True)
    
card_row = df_m.groupby(key).size()
card_row.name = 'index'
ohe = ohe.join(card_row)

for col in ohe.columns:
    if col.count(feat_col):
        ohe[col] = ohe[col]/ohe['index']
        
ohe.head()

,state_id_-1,state_id_1,state_id_2,state_id_3,state_id_4,state_id_5,state_id_6,state_id_7,state_id_8,state_id_9,state_id_10,state_id_11,state_id_12,state_id_13,state_id_14,state_id_15,state_id_16,state_id_17,state_id_18,state_id_19,state_id_20,state_id_21,state_id_22,state_id_23,state_id_24,index
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,0.185430,0.0,0.801325,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.006623,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.006623,0.0,0.0,0.000000,151
C_ID_0001238066,0.073826,0.0,0.000000,0.000000,0.0,0.006711,0.0,0.000000,0.0,0.751678,0.0,0.000000,0.0,0.0,0.0,0.020134,0.0,0.0,0.000000,0.000000,0.14094,0.006711,0.0,0.0,0.000000,149
C_ID_0001506ef0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.029412,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.970588,0.00000,0.000000,0.0,0.0,0.000000,68
C_ID_0001793786,0.469636,0.0,0.000000,0.000000,0.0,0.004049,0.0,0.000000,0.0,0.105263,0.0,0.080972,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.340081,247
C_ID_000183fdda,0.025806,0.0,0.000000,0.890323,0.0,0.006452,0.0,0.019355,0.0,0.045161,0.0,0.006452,0.0,0.0,0.0,0.000000,0.0,0.0,0.006452,0.000000,0.00000,0.000000,0.0,0.0,0.000000,155


In [38]:
base = utils.read_df_pkl('../input/base*')
base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)

df_train = base_train.merge(ohe, how='left', on=key)
df_test = base_test.merge(ohe, how='left', on=key)

for col in df_train.columns:
    if col in ignore_list or col.count('index'):continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/115_out_train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/115_out_test_{col}@', obj=df_test[col].values)

100%|██████████| 3/3 [00:00<00:00, 33.54it/s]
